<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/verbs_cosine_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz


--2020-02-17 07:30:17--  http://rusvectores.org/static/models/rusvectores2/news_mystem_skipgram_1000_20_2015.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549952184 (524M) [application/x-gzip]
Saving to: ‘news_mystem_skipgram_1000_20_2015.bin.gz’

news_mystem_skipgra 100%[===================>] 524.47M  27.9MB/s    in 20s     

2020-02-17 07:30:37 (26.4 MB/s) - ‘news_mystem_skipgram_1000_20_2015.bin.gz’ saved [549952184/549952184]



In [2]:
!rm collection.json.1 collection.json
!wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json

rm: cannot remove 'collection.json.1': No such file or directory
rm: cannot remove 'collection.json': No such file or directory
--2020-02-17 07:30:43--  https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 606414 (592K) [text/plain]
Saving to: ‘collection.json’

collection.json     100%[===================>] 592.20K  --.-KB/s    in 0.09s   

2020-02-17 07:30:44 (6.36 MB/s) - ‘collection.json’ saved [606414/606414]



In [0]:
import gensim
from collections import Counter
import json
import pandas as pd

In [4]:
rv_model = gensim.models.KeyedVectors.load_word2vec_format('news_mystem_skipgram_1000_20_2015.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# rv_model.vocab
# rv_model.get_vector('помочь_V')

In [5]:

verbs_for_check_dict = {'снимать': [['a0', 'a1', 'neg', 1.0]]}

def just_words_similarity(base_verb, verbs_for_check_dict):
  """
  Функция тестовая, проверяет, как работает косинусная близость.
  """
  group_dict = Counter()
  my_base_verb = base_verb + '_V'
  for verb in verbs_for_check_dict.keys():
    verb_for_check = verb + '_V'
    similarity = rv_model.similarity(my_base_verb, verb_for_check)
    group_dict[str(verbs_for_check_dict[verb])] = [base_verb, verb, similarity]
  return group_dict

just_words_similarity('радоваться', verbs_for_check_dict)




/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Counter({"[['a0', 'a1', 'neg', 1.0]]": ['радоваться', 'снимать', 0.15058792]})

In [0]:
# rv_model.vocab

In [0]:
collection_handler = open("collection.json", "r", encoding="utf-8")
collection = json.load(collection_handler, encoding="utf-8")

In [134]:

def make_group(polarity_type_list):
  """
  На вход подаются пары из разных типов отношений(p_type) в списках(как в переменной pol_type_dict)
  для каждой пары цикл ищет не более ста глаголов, у которых есть типы отношений, совпадающие с нашей парой, 
  которая подается на вход. 
  """
  group_dict = Counter()
  for p_type in polarity_type_list:
    print(p_type)
    count = 0
    for key in collection.keys():
      if count < 100:
        try:
          if p_type[0] in collection[key]['frames']['polarity'] and p_type[1] in collection[key]['frames']['polarity'] :
            local_verb = collection[key]['title'][0]
            if  rv_model.vocab[local_verb + '_V']: #проверка, есть-ли в модели глагол
              group_dict[local_verb] = p_type
              count += 1
            for variant in collection[key]['variants']:
              if rv_model.vocab[variant + '_V']: #проверка, есть-ли в модели глагол
                group_dict[variant] = p_type
                count += 1
        except Exception as e:
          pass
  return group_dict

pol_type_dict = [[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]], \
                 [['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]], \
                 [['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]], \
                 [['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]], \
                 [['a1', 'a2', 'pos', 1.0], ['a2', 'a1', 'pos', 1.0]], \
                 [['a1', 'a2', 'neg', 1.0], ['a2', 'a1', 'neg', 1.0]]]

groups_total_dict = make_group(pol_type_dict)
len(groups_total_dict)
# make_group(pol_type_dict)


[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos', 1.0]]
[['a0', 'a1', 'pos', 0.7], ['a1', 'a0', 'pos', 0.7]]
[['a0', 'a1', 'neg', 0.7], ['a1', 'a0', 'neg', 0.7]]
[['a0', 'a1', 'neg', 1.0], ['a1', 'a0', 'neg', 1.0]]
[['a1', 'a2', 'pos', 1.0], ['a2', 'a1', 'pos', 1.0]]
[['a1', 'a2', 'neg', 1.0], ['a2', 'a1', 'neg', 1.0]]


80

In [135]:
def make_similarity_df(base_verb, verbs_for_check_dict):
  '''
  Считает косинусное расстояние с каким-либо подающимся на вход сидовым глаголом глаголов из словаря, полученного с помощью 
  функции make_group. Возвращает словарь с ключами 'groups', 'base_verb', 'verb' и 'verbs_similarity', 
  которые потом будут колонками таблицы - датафрейма.
  '''

  df_dict = Counter()
  df_dict['groups'] = []
  df_dict['base_verb'] = []
  df_dict['verb'] = []
  df_dict['verbs_similarity'] = []

  my_base_verb = base_verb + '_V'
  for verb in verbs_for_check_dict.keys():
    verb_for_check = verb + '_V'
    try:
      verbs_similarity = rv_model.similarity(my_base_verb, verb_for_check)
      df_dict['groups'].append(str(verbs_for_check_dict[verb]))
      df_dict['base_verb'].append(base_verb)
      df_dict['verb'].append(verb)
      df_dict['verbs_similarity'].append(verbs_similarity)
    except KeyError as e:
      pass
      print(e)
  return df_dict

verbs_for_df = make_similarity_df('радоваться', groups_total_dict)
print('done')

done


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:

frames_df = pd.DataFrame(verbs_for_df, columns = verbs_for_df.keys()) # тут делается датафрейм

In [133]:
frames_df

,groups,base_verb,verb,verbs_similarity
0,"[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos',...",радоваться,приветствовать,0.239950
1,"[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos',...",радоваться,здороваться,0.298290
2,"[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos',...",радоваться,радовать,0.369049
3,"[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos',...",радоваться,популяризовать,0.120459
4,"[['a0', 'a1', 'pos', 1.0], ['a1', 'a0', 'pos',...",радоваться,дарить,0.273577
...,...,...,...,...
75,"[['a1', 'a2', 'pos', 1.0], ['a2', 'a1', 'pos',...",радоваться,ревновать,0.289864
76,"[['a1', 'a2', 'neg', 1.0], ['a2', 'a1', 'neg',...",радоваться,пострадать,0.088961
77,"[['a1', 'a2', 'neg', 1.0], ['a2', 'a1', 'neg',...",радоваться,страдать,0.193153
78,"[['a1', 'a2', 'neg', 1.0], ['a2', 'a1', 'neg',...",радоваться,терпеть,0.224986
